# Seminar 1

# OpenAI Gym. Value Iteration. Policy Iteration. Tabular Q-learning

## Введение в OpenAI Gym

 <a href=https://gym.openai.com>OpenAI Gym</a> это набор инструментов для разработки и сравнения алгоритмов обучения с подкреплением.

OpenAI Gym предоставляет простой и универсальный API ко многим средам с разными свойствами, как простым так и сложным:
* Классические задачи управления и игрушечные примеры, которые можно найти в учебниках и на которых демонстрируется работа алгоритмов обучения с подкреплением (в основном они будут использоваться в этом курсе)
* Игры Atari (оказали огромное влияние на достижения в обучении с подкреплением в последние годы)
* 2D и 3D среды для контроля роботов в симуляции (используют проприетарный движок <a href=http://www.mujoco.org/>MuJoCo</a>)

#### Для работы нам потребуется установить gym и numpy. Gym следует установить командой pip install gym=0.9.2

In [21]:
!pip install gym -U

Requirement already up-to-date: gym in /usr/local/lib/python3.7/site-packages (0.15.4)


In [22]:
!pip install opencv

ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


In [23]:
# Импортируем необходимые библиотеки
import gym
import numpy as np
import itertools
from collections import defaultdict
from gym.envs import toy_text
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [24]:
env = gym.make('CartPole-v0') # создаем среду CartPole-v0

[2019-11-16 10:23:15,009] Making new env: CartPole-v0
/usr/local/lib/python3.7/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [25]:
env.reset()
for _ in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample()) # агент выбирает случайные действия
    if done:
        env.reset()

ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [26]:
env.close() # выключим визуализацию

Вспомним элементы проблемы обучения с подкреплением <img src="./scheme.png" width=500>

OpenAI Gym предоставляет такой же интерфейс взаимодействия со средой:
* Среда в ответ на действие агента предоставляет *bservation (object)*- специфичный для конкретной среды объект, который предствляет наблюдения агента. Например, пиксели камеры, значения углов и скоростей сочленений робота или позиции агента и других объектов в среде.
* *reward (float)* - значение награды, полученной агентом в резултате совершенного действия
* *done (boolean)* - флаг обозначающий окончание эпизода. Например, эпизод заканчивается, когда шест слишком сильно отклонился или агент попал в прорубь в среде FrozenLake
* *info (dict)* - словарь, содержащий диагностическую информацию, которую можно использовать для отладки, но не для обучения агента. Обычно мы присваеваем значение *info* переменной по-умолчению *_*

Теперь, когда мы познакомились с API OpenAI Gym, посмотрим, сколько очков награды сможет в среднем получить за 100 эпизодов агент, выбирающий случайные действия.

In [27]:
total_reward = []
env = gym.make('CartPole-v0')
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

[2019-11-16 10:23:20,265] Making new env: CartPole-v0
/usr/local/lib/python3.7/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [ ]:
env.close()

In [ ]:
print(np.mean(total_reward))

Наш "cлучайный" агент получает в среднем 20 очков за 100 эпизодов. Не очень впечатляет(

В предыдущем эксперименте агент выбирал случайное действие. Важными объектами в OpenAI Gym являются пространства состояний и действий.

In [28]:
env = gym.make('CartPole-v0')

[2019-11-16 10:23:36,876] Making new env: CartPole-v0
/usr/local/lib/python3.7/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [29]:
print(env.action_space.__doc__)


    {0,1,...,n-1}

    Example usage:
    self.observation_space = spaces.Discrete(2)
    


In [30]:
print(env.action_space)

Discrete(2)


In [31]:
print(env.action_space.n)

2


In [32]:
print(env.observation_space.__doc__)


    A box in R^n.
    I.e., each coordinate is bounded.

    Example usage:
    self.action_space = spaces.Box(low=-10, high=10, shape=(1,))
    


In [33]:
print(env.observation_space)

Box(4,)


In [34]:
print(env.observation_space.shape)

(4,)


In [35]:
env.observation_space.high

array([4.80000000e+00, 3.40282347e+38, 4.18879020e-01, 3.40282347e+38])

In [36]:
env.observation_space.low

array([-4.80000000e+00, -3.40282347e+38, -4.18879020e-01, -3.40282347e+38])

## Value Iteration

На лекции мы рассмотрели, как мы можем выучить оптимальную политику, используя алгоритм Value Iteration, если нам известна динамика среды, а также если пространства состояний и действий не большие и дискретные.

Попробуем выучить оптимальную политику в среде <a href=https://gym.openai.com/envs/FrozenLake-v0>FrozenLake-v0</a>. Это простая среда с маленькими пространствами состояний и действий, а также с известной динамикой.

Создадим среду и выведем её описание.

In [37]:
env = gym.make('FrozenLake-v0')

[2019-11-16 10:23:40,598] Making new env: FrozenLake-v0


In [38]:
print(env.env.__doc__)


    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.

    


Как видно среда представляет собой поле 4 на 4, по которому нужно добраться от начала (клетка *S*) до цели (клетка *G*). При этом среда является недетерменированный - с определенной вероятностью при совершения действия агент подскользнется и попадет не в ту клетку, в которую направлялся. Клетка *H* обозначает прорубь. Игра закначивается, когда агент попадает в клетку *G* или в клету *H*. Если агент проваливается в прорубь, то он получает награду *0*, если достигает клетки цели - *1*. 

Посмотрим, сколько в среднем очков награды за 100 эпизодов получит наш агент, если будет выполнять случайные действия.

In [39]:
env.seed(0); from gym.spaces import prng; prng.seed(10) # установим сид для воспроизводимости результатов эксперимента

In [40]:
total_reward = []
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Episode 1 finished after 2 timesteps

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Episode 2 finished after 11 timesteps

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode 3 finished after 2 timesteps

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Episode 4 finished after 2 timesteps

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode 5 finished after 2 timesteps

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
Episode 6 finished after 3 timesteps

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
H

In [41]:
print(np.mean(total_reward))

0.03


Как видим, только в 3 эпизодах из 100 агену удалось добраться до цели.

Из среды OpenAI Gym мы можем получить элементы MDP (Markov Decision Process).

В env.env.P хранится двухуровневый словарь, в котором первый ключ является состояние, а второй - действием.
Клетки ассоциированыс индексами [0, 1, 2, ..., 15] слева направо и сверху вниз.

Индексы действией [0, 1, 2, 3] соответствуют движению на Запад, Юг, Восток и Север.
env.env.P[state][action] возвращает лист кортежей (probability, nextstate, reward). Например, состояние 0 - это начальное состояние и информация о веротностях перехода для s=0 и a=0 содержит:

In [42]:
env.env.P[0][0]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

Другой пример - состояние 5 сооветсвует проруби, и все действия в данном состоянии приводят к тому же состоянию с вероятностью 1 и наградой 0.

In [43]:
for i in range(4):
    print("P[5][%i] =" % i, env.env.P[5][i])

P[5][0] = [(1.0, 5, 0, True)]
P[5][1] = [(1.0, 5, 0, True)]
P[5][2] = [(1.0, 5, 0, True)]
P[5][3] = [(1.0, 5, 0, True)]


Вспомним, что из себя представляет алгоритм Value Iteration <img src="./value_iteration.png" width="500">

Задание считается решенным, если агент доходит до цели в среднем в 70% эпизодов.

Напишем несклько вспомогательных функций.

Запомним число стостояний и действий в среде.

In [44]:
n_states = env.env.nS
n_actions = env.env.nA
print("Number of states: {}".format(n_states))
print("Number of actions: {}".format(n_actions))

Number of states: 16
Number of actions: 4


Поскольку алгоритм Value Iteration возвращает нам оптимальную V-функцию, то нам необходимо извлекать из нее оптимальную политику (как указано в последней строке псевдокода алгоритма).

In [45]:
def extract_policy(v, gamma = 1.0):
    policy = np.zeros(n_states)
    for state in range(n_states):
        q = np.zeros(n_actions)
        for action in range(n_actions):
            for next_sr in env.env.P[state][action]:
                probability, next_state, reward, _ = next_sr
                q[action] += (probability*(reward + gamma*v[next_state]))
        policy[state] = np.argmax(q)
    return policy

Также напишем функцию для оценки нашей найденной политики.

In [46]:
def evaluate_policy(env, policy, gamma=1.0, n=100):
    total_reward = []
    for episode in range(n):
        episode_reward = 0
        observation = env.reset()
        step = 0
        for _ in range(100):
            env.render()
            action = int(policy[observation])
            observation, reward, done, _ = env.step(action)
            episode_reward += gamma**step*reward
            step += 1
            if done:
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Нам остается написать основную функцию, которая вернет оптимальную V-функцию.

In [47]:
def value_iteration(env, gamma=1.0, max_iterations = 100000):
    v = np.zeros(n_states)
    eps = 1e-20
    
    
    
    for s in range(n_states):
        
        
    
    return v

Теперь мы можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [ ]:
optimal_v = value_iteration(env)
optimal_policy = extract_policy(optimal_v)
optimal_policy_score = evaluate_policy(env, optimal_policy, n=100)

In [ ]:
print(optimal_v.reshape(4,4))

In [ ]:
print(optimal_policy.reshape(4,4))

In [ ]:
print(optimal_policy_score)

По сравнению со "случайным" агентом, который доходил до цели в 3 случаях из 100, наша новая политика позволяет добирться до цели в ~70% эпизодов.

## Policy Iteration

Вспомним, что из себя представляет алгоритм Policy Iteration <img src="policy_iteration.png" width="500">

Напишем необходимые вспомогательные функции.

Начнем с основного цикла алгоритма, который вернет нам оптимальную политику.

In [ ]:
def policy_iteration(env, gamma=1.0, max_iterations = 200000):
    policy = np.random.choice(n_actions, size=(n_states))  # initialize a random policy
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

Остается написать 2 функции, которые используются в основном цикле алгоритма Policy Iteration согласно псевдокоду.

In [ ]:
def compute_policy_v(env, policy, gamma=1.0):
    v = np.zeros(n_states)
    eps = 1e-10
    # Your code goes here
    return v

In [ ]:
def extract_policy(v, gamma=1.0):
    policy = np.zeros(n_states)
    # Your code goes here
    return policy

Теперь мы также можем найти оптимальную V-функцию, извлечь из нее оптимальную политику и оцениь ее.

In [ ]:
optimal_policy = policy_iteration(env)
optimal_policy_score = evaluate_policy(env, optimal_policy)

In [ ]:
print(optimal_policy.reshape(4,4))

In [ ]:
print(optimal_policy_score)

## Tabular Q-learning

Реализуем алгоритм Q-learning для среды CliffWalking. 

Посмотрим, что из себя представляет среда CliffWalking <img src="cliffwalking.png" width="500">

In [ ]:
env = toy_text.CliffWalkingEnv()

In [ ]:
print(env.__doc__)

Так как награда выражатеся отрицательными значениями, то фактической целью агента является как можно более быстрое преодоление пути от старта к финишу при этом ему нужно не упасть с обрыва.

Традиционно посмотрим сколько очков в среднем за 100 эпизодов сможет набрать "cлучайный" агент.

In [ ]:
total_reward = []
for episode in range(100):
    episode_reward = 0
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print("Episode {} finished after {} timesteps".format(episode+1, t+1))
            break
    total_reward.append(episode_reward)

In [ ]:
print(np.mean(total_reward))

Ожидаемо, наш "случайный" агент просто блуждает по среде, не пытаясь добраться до цели.

Вспомним как выглядит реализация данного алгоритма <img src="q_learning.png" width="500">

Так как мы раелизуем табличную версию Q-learning, создадим структура, которая будет хранить значения нашей функции *Q(S,A)* для каждого состояния и действия. Она пдетставляет собой словарь (*dict*), хранящий в качестве ключей состояния, а в качестве значений массив значений *Q-функции* для каждого действия для данного ключа-состояния.

In [ ]:
Q = defaultdict(lambda: np.zeros(env.action_space.n))

print("Q-значения для состояния-действия (0, 0): %s" % Q[(0, 0)], "хранятся в списке, по значению для каждого действия")
print("Таким обарзом, Q-значение для действия 3 в в состоянии (1,2), i.e. Q((1,2), 3), можно получить вот так q_vals[(1,2)][3]:", Q[(1,2)][3])

Сначала напишем функцию реализующую $\epsilon$-greedy политику для исследования среды.

In [ ]:
def eps_greedy(Q, epsilon, state):
    """
    Параметры:
        Q: таблица значений Q-функции
        epsilon: параметр эпсилон
        state: текущее состоние
    Результат:
        Случайное действие с вероятностью eps или argmax Q(s, .) c вероятностью (1 - eps)
        random action with probability of eps; argmax Q(s, .) with probability of (1-eps)
    """
    # To Do

Также напишем функцию, которая поможет нам оценить, как ведет себя обученный агент.

In [ ]:
def q_eval(Q, render=True):
    total_reward = []
    for i_episode in range(100):
        episode_reward = 0
        observation = env.reset()
        for t in range(100):
            if render:
                env.render()
            action = np.argmax(Q[observation])
            observation, reward, done, info = env.step(action)
            episode_reward += reward
            if done:
                print("Episode {} finished after {} timesteps".format(i_episode+1, t+1))
                break
        total_reward.append(episode_reward)
    return np.mean(total_reward)

Теперь реализуем функцию обучения агента (обновления значений Q-функции) с использование функции, реализующей $\epsilon$-greedy политику.

In [ ]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """    
    Параменты:
        env: среда Open AI
        num_episodes: Количество эпизодов
        discount_factor: фактор дисконтирования
        alpha: константа обучения
        epsilon: параметр эпсилон для ϵ-greedy политику
    
    Результат:
        Таблица с оптимальными значениями Q-функции
    """
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    episode_lengths=np.zeros(num_episodes)
    episode_rewards=np.zeros(num_episodes)
        
    for i_episode in range(num_episodes):
        if (i_episode + 1) % 100 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
        
        state = env.reset()
        
        # To Do
    
    return Q, episode_lengths, episode_rewards

In [ ]:
env = toy_text.CliffWalkingEnv()

In [ ]:
Q, episode_lengths, episode_rewards = q_learning(env, 500)

In [ ]:
score = q_eval(Q)

In [ ]:
plt.plot(episode_lengths)

In [ ]:
plt.plot(episode_rewards)